# Experiments with using Instructor JS and our decision service api

* Trying to figure out what schemas / types info the API should provide, so that a consumer can use one of these schema/grammar x llm libraries easily with the API
* Should also be helpful to just see how easy it is to use one of these popular libraries with the API
* A notebook like this should also make it easier for other people to run their own experiments

In [10]:
process.env.envPath = '../../.env.local'

import { APIClient } from '../lib/decision-api-client/api.ts'
import { appConfig } from '../config.ts'


const client = await APIClient.make(appConfig)


"../../.env.local"

## Some GETs, to see what fns are currently loaded, and to check that the configuration is OK

In [11]:
await client.getSimpleFunctionEndpoints()

[
  {
    function: {
      description: "Determines if a person qualifies for the purposes of the rule.\n" +
        "The input object describes the person's properties in the primary parameters: walks, eats, drinks.\n" +
        "Secondary parameters can be given which are sufficient to determine some of the primary parameters.\n" +
        "A person drinks whether or not they consume an alcoholic or a non-alcoholic beverage, in part or in whole;\n" +
        "those specific details don't really matter.\n" +
        "The output of the function can be either a request for required information;\n" +
        "a restatement of the user input requesting confirmation prior to function calling;\n" +
        "or a Boolean answer with optional explanation summary.",
      name: "compute_qualifies"
    },
    type: "function"
  },
  {
    function: {
      description: "We do not cover any loss or damage caused by rodents, insects, vermin, or birds.\n" +
        "However, this exclusion does n

In [22]:
const detailedEndpointMap = await client.getDetailedEndpointInfoMap()
detailedEndpointMap

Map(2) {
  "compute_qualifies" => {
    function: {
      description: "Determines if a person qualifies for the purposes of the rule.\n" +
        "The input object describes the person's properties in the primary parameters: walks, eats, drinks.\n" +
        "Secondary parameters can be given which are sufficient to determine some of the primary parameters.\n" +
        "A person drinks whether or not they consume an alcoholic or a non-alcoholic beverage, in part or in whole;\n" +
        "those specific details don't really matter.\n" +
        "The output of the function can be either a request for required information;\n" +
        "a restatement of the user input requesting confirmation prior to function calling;\n" +
        "or a Boolean answer with optional explanation summary.",
      name: "compute_qualifies",
      parameters: {
        properties: { drinks: [Object], eats: [Object], walks: [Object] },
        type: "object"
      },
      supportedBackends: []
    },
    t

In [23]:
detailedEndpointMap.get("vermin_and_rodent").function.parameters

{
  properties: {
    "Loss or Damage.caused by birds": {
      alias: null,
      description: "Was the damage caused by birds?",
      enum: [ "true", "false" ],
      type: "string"
    },
    "Loss or Damage.caused by insects": {
      alias: null,
      description: "Was the damage caused by insects?",
      enum: [ "true", "false" ],
      type: "string"
    },
    "Loss or Damage.caused by rodents": {
      alias: null,
      description: "Was the damage caused by rodents?",
      enum: [ "true", "false" ],
      type: "string"
    },
    "Loss or Damage.caused by vermin": {
      alias: null,
      description: "Was the damage caused by vermin?",
      enum: [ "true", "false" ],
      type: "string"
    },
    "Loss or Damage.ensuing covered loss": {
      alias: null,
      description: "Is the damage ensuing covered loss",
      enum: [ "true", "false" ],
      type: "string"
    },
    "Loss or Damage.to Contents": {
      alias: null,
      description: "Is the damage to yo

### Try a quick simple example with Instructor

In [24]:
import Instructor from "@instructor-ai/instructor";
import OpenAI from "openai"
import { z } from "zod"

const oai = new OpenAI({
    apiKey: appConfig.getLlmApiKey(),
    organization: process.env.OPENAI_ORG_ID ?? undefined
  })

const instrClient = Instructor({
  client: oai,
  mode: "TOOLS"
})

In [25]:
import { makeFunctionParamSchema } from '../lib/query/query-schemas.ts'

const quickVerminPrompt = "I'm trying to figure out if I can claim for damage to my AC -- a rat crawled into it, did some ratty things, and now it no longer works. I am able to call a function that can help me check this. Here's the parameters for said function. Please construct json parameters for this function that I can use to call it."

const verminFnInfo = await client.getInfoFor("vermin_and_rodent")
const VerminParamSchema = makeFunctionParamSchema(verminFnInfo)

const sampleVerminExtractParams: z.infer<typeof VerminParamSchema> = await instrClient.chat.completions.create({
    messages: [{ role: "user", content: quickVerminPrompt }],
    model: "gpt-4o",
    response_model: { 
      schema: VerminParamSchema, 
      name: "ExtractParams"
    }
  })

sampleVerminExtractParams

{
  "Loss or Damage.caused by birds": "false",
  "Loss or Damage.caused by insects": "false",
  "Loss or Damage.caused by rodents": "true",
  "Loss or Damage.caused by vermin": "false",
  "Loss or Damage.ensuing covered loss": "false",
  "Loss or Damage.to Contents": "false",
  "a household appliance": "false",
  "a plumbing, heating, or air conditioning system": "true",
  "a swimming pool": "false",
  "any other exclusion applies": "false",
  _meta: {
    usage: {
      prompt_tokens: 347,
      completion_tokens: 93,
      total_tokens: 440,
      prompt_tokens_details: { cached_tokens: 0, audio_tokens: 0 },
      completion_tokens_details: {
        reasoning_tokens: 0,
        audio_tokens: 0,
        accepted_prediction_tokens: 0,
        rejected_prediction_tokens: 0
      }
    },
    thinking: undefined
  }
}

In [26]:
// Try calling the dec service API

await client.evalFunctionEndpoint(endpointNameToExperimentWith, finalSampleVerminExtractParams as z.infer<typeof VerminParamSchema>)


{
  ok: {
    contents: {
      reasoning: { payload: { children: [Array], payload: [Object] } },
      values: [ [ "result", "TRUE" ] ]
    },
    tag: "SimpleResponse"
  },
  error: undefined
}